<a href="https://colab.research.google.com/github/erickrribeiro/ufam_nlp_01_2025/blob/main/scripts/fase-03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Obrigatório para testes no Google Colab, senão ignorar
from google.colab import userdata

github_token = userdata.get('GITHUB_TOKEN')
!git clone https://{github_token}@github.com/erickrribeiro/ufam_nlp_01_2025.git
!mv ./ufam_nlp_01_2025/scripts/** ./
!cp ./ufam_nlp_01_2025/models/* ./
!cp ./ufam_nlp_01_2025/custom_metrics/* ./
!unzip ./mistral-sql-lora-v3.zip -d ./
!unzip ./mistral-sql-lora-v4.zip -d ./

Cloning into 'ufam_nlp_01_2025'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 32 (delta 7), reused 12 (delta 4), pack-reused 16 (from 1)
Receiving objects: 100% (32/32), 12.24 MiB | 32.65 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Archive:  ./mistral-sql-lora-v3.zip
  inflating: ./mistral-sql-lora-v3/adapter_config.json  
  inflating: ./mistral-sql-lora-v3/README.md  
  inflating: ./mistral-sql-lora-v3/adapter_model.safetensors  
Archive:  ./mistral-sql-lora-v4.zip
  inflating: ./mistral-sql-lora-v4/README.md  
  inflating: ./mistral-sql-lora-v4/adapter_config.json  
  inflating: ./mistral-sql-lora-v4/adapter_model.safetensors  


In [3]:
!rm -rf ./ufam_nlp_01_2025
# !rm -rf ./spider
!rm -rf mistral-sql-lora-*
!rm *.py
!rm *.ipynb
!rm *.csv
!rm *.zip

In [3]:
!pip install -U gdown==5.2.0 \
                transformers==4.52.4 \
                trl==0.18.2 \
                peft==0.15.2 \
                datasets==3.6.0 \
                accelerate==1.8.0 \
                huggingface_hub==0.33.0 \
                fsspec==2025.3.0 \
                deepeval==3.1.6 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [38]:
from utils import download_spider_dataset

download_spider_dataset()

Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=0aee2a1e-97c5-483c-a9d4-e54aa2b26271
To: /content/spider_data.zip
100%|██████████| 206M/206M [00:02<00:00, 89.5MB/s]


In [39]:
!unzip spider_data.zip -d spider

Archive:  spider_data.zip
   creating: spider/spider_data/
  inflating: spider/spider_data/dev_gold.sql  
  inflating: spider/__MACOSX/spider_data/._dev_gold.sql  
   creating: spider/spider_data/database/
  inflating: spider/__MACOSX/spider_data/._database  
  inflating: spider/spider_data/.DS_Store  
  inflating: spider/__MACOSX/spider_data/._.DS_Store  
  inflating: spider/spider_data/test_tables.json  
  inflating: spider/__MACOSX/spider_data/._test_tables.json  
  inflating: spider/spider_data/train_others.json  
  inflating: spider/__MACOSX/spider_data/._train_others.json  
  inflating: spider/spider_data/train_spider.json  
  inflating: spider/__MACOSX/spider_data/._train_spider.json  
  inflating: spider/spider_data/test.json  
  inflating: spider/__MACOSX/spider_data/._test.json  
  inflating: spider/spider_data/tables.json  
  inflating: spider/__MACOSX/spider_data/._tables.json  
  inflating: spider/spider_data/dev.json  
  inflating: spider/__MACOSX/spider_data/._dev.json  

In [4]:
import datasets, transformers, trl, peft, accelerate, huggingface_hub, fsspec, deepeval

print("datasets:", datasets.__version__)
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("trl:", trl.__version__)
print("accelerate:", accelerate.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("fsspec:", fsspec.__version__)
print("deepeval:", deepeval.__version__)

datasets: 3.6.0
transformers: 4.52.4
peft: 0.15.2
trl: 0.18.2
accelerate: 1.8.0
huggingface_hub: 0.33.0
fsspec: 2025.3.0
deepeval: 3.1.6


# Fase 3: Avaliação de Desempenho na Tarefa-Alvo com Métrica Customizada


## 3.1. Implementação da Métrica:

A seguri será desenvolvida uma métrica customizada em DeepEval para "Execution Accuracy". A classe da métrica deve seguir a seguinte estrutura:

- Herdar de deepeval.metrics.BaseMetric.
- Implementar o método measure(self, test_case: LLMTestCase) -> float.
- A lógica interna do measure deve:
    - a. Estabelecer uma conexão com um banco de dados sqlite contendo a base de dados de teste do Spider.
    - b. Executar a consulta SQL contida em test_case.actual_outputdentro de uma transação segura (para lidar com erros de sintaxe via try-except).
    - c. Executar a consulta ground truthcontida em test_case.expected_output.
    - d. Comparar os conjuntos de resultados. A comparação deve ser insensível à ordem das linhas. e. Retornar 1.0 para sucesso (resultados idênticos) e 0.0 para falha.

## Testando a classe

In [40]:
from deepeval import assert_test, evaluate
from deepeval.evaluate.configs import AsyncConfig
from deepeval.test_case import LLMTestCase
from accuracy_metrics import ExecutionAccuracyMetric

metric = ExecutionAccuracyMetric()

test_cases = [
  LLMTestCase(
    input="List the name, born state and age of the heads of departments ordered by age.",
    actual_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gerado pelo modelo
    expected_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gold
    additional_metadata={"db_id": "department_management"}
  ),
  LLMTestCase(
    input="List the name, born state and age of the heads of departments ordered by age.",
    actual_output="SSSELECT born_state ,  age FROM head ORDER BY age;",  # SQL gerado pelo modelo
    expected_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gold
    additional_metadata={"db_id": "department_management"}
  )
]
result = evaluate(
  test_cases=test_cases,
  metrics=[metric],
  async_config=AsyncConfig(run_async=False)
)

✨ You're running DeepEval's latest AccuracyMetric (NLP-TP-04) Metric! (using None, strict=False, 
async_mode=False)...

Output()



Metrics Summary

  - ✅ AccuracyMetric (NLP-TP-04) (score: 1.0, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: None)

For test case:

  - input: List the name, born state and age of the heads of departments ordered by age.
  - actual output: SELECT name ,  born_state ,  age FROM head ORDER BY age;
  - expected output: SELECT name ,  born_state ,  age FROM head ORDER BY age;
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

AccuracyMetric (NLP-TP-04): 50.00% pass rate




Metrics Summary

  - ❌ AccuracyMetric (NLP-TP-04) (score: None, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: [Error] Failed executing predicted SQL: near "SSSELECT": syntax error)

For test case:

  - input: List the name, born state and age of the heads of departments ordered by age.
  - actual output: SSSELECT born_state ,  age FROM head ORDER BY age;
  - expected output: SELECT name ,  born_state ,  age FROM head ORDER BY age;
  - 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

# Avaliando o Baseline

In [41]:
!deepeval test --help

                                                                                
 Usage: deepeval test [OPTIONS] COMMAND [ARGS]...                               
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                  │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ run   Run a test                                                             │
╰──────────────────────────────────────────────────────────────────────────────╯



In [2]:
!deepeval test run test_baseline.py

A saída de streaming foi truncada nas últimas 5000 linhas.
  - actual output: SELECT c.Name
FROM conductor c
JOIN orchestra o ON c.Conductor_ID = o.Conductor_ID
WHERE o.Year_of_Founded > 2008;
  - expected output: SELECT T1.Name FROM conductor AS T1 JOIN orchestra AS T2 ON T1.Conductor_ID  =  T2.Conductor_ID WHERE Year_of_Founded  >  2008
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

AccuracyMetric (NLP-TP-04): 42.94% pass rate




Metrics Summary

  - ✅ AccuracyMetric (NLP-TP-04) (score: 1.0, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: None)

For test case:

  - input: What are the names of conductors who have conducted orchestras founded after the year 2008?
  - actual output: SELECT c.Name
FROM conductor c
JOIN orchestra o ON c.Conductor_ID = o.Conductor_ID
WHERE o.Year_of_Founded > 2008;
  - expected output: SELECT T1.Name FROM conductor AS T1 JOIN orchestra AS T2 ON T1.Conductor_ID  =  T2.Conductor_ID WHERE Year_of_Found

In [6]:
!deepeval test run test_mistral_sql_lora_v3.py

A saída de streaming foi truncada nas últimas 5000 linhas.
For test case:

  - input: What are the country codes of the different countries, and what are the languages spoken by the greatest percentage of people for each?
  - actual output: SELECT CountryCode, Language FROM countrylanguage ORDER BY Percentage DESC
  - expected output: SELECT LANGUAGE ,  CountryCode ,  max(Percentage) FROM countrylanguage GROUP BY CountryCode
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

AccuracyMetric (NLP-TP-04): 43.13% pass rate




Metrics Summary

  - ❌ AccuracyMetric (NLP-TP-04) (score: 0.0, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: None)

For test case:

  - input: What is the total number of countries where Spanish is spoken by the largest percentage of people?
  - actual output: SELECT COUNT(*) FROM countrylanguage WHERE CountryCode IN (SELECT CountryCode FROM countrylanguage ORDER BY Percentage DESC LIMIT 1) AND Language =  'Spanis

In [7]:
!deepeval test run test_mistral_sql_lora_v4.py

A saída de streaming foi truncada nas últimas 5000 linhas.
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:00
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:00
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━━━━╺━━━━━━━━━━━━━  45% 0:02:01
Evaluating 1034 test case(s) sequentially ━━━━━━━━